In [1]:
import pickle
import cv2
import numpy as np
import os
import re
import time
import warnings
import numpy as np
from skimage.feature import hog
from sklearn.metrics import accuracy_score


In [2]:
def preprocess_image(image):
        # Convert the image to YCrCb color space
        image_ycrcb = cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)

        # Split the image into channels
        y, cr, cb = cv2.split(image_ycrcb)

        # Apply thresholding to the Cr channel using Otsu's thresholding
        _, cr_thresholded = cv2.threshold(cr, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

       # Apply Gaussian blur multiple times
        for _ in range(5):
            image_filtered = cv2.GaussianBlur(cr_thresholded, (5, 5), 0)
            cr_thresholded=image_filtered

        # Define the kernel for morphological operations
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15))  # Elliptical kernel
        
        # Perform morphological opening multiple times
        iterations = 10
        for _ in range(iterations):
            opened = cv2.morphologyEx(image_filtered, cv2.MORPH_OPEN, kernel)
            # Update the image for the next iteration
            image_filtered = opened
    
        #cv2.imwrite(output_path, opened)
        return opened

In [3]:
# Feature Extraction Module
def extract_features(image):
    # Extract HOG features
    hog_features = hog(image, orientations=9, pixels_per_cell=(12, 12), cells_per_block=(2, 2), visualize=False)
    # Calculate Hu moments
    moments = cv2.moments(image)
    hu_moments = cv2.HuMoments(moments).flatten()
    features = np.concatenate((hog_features, hu_moments))
    return features

In [4]:
# Suppress the UserWarning
warnings.filterwarnings("ignore", category=UserWarning)
# Load the model from the Pickle file
with open("./classifiers/Random_Forest.pickle", "rb") as file:
    model_1 = pickle.load(file)



In [5]:
# Suppress the UserWarning
warnings.filterwarnings("ignore", category=UserWarning)
# Load the model from the Pickle file
with open("./classifiers/K-Nearest Neighbors.pickle", "rb") as file:
    model_2 = pickle.load(file)


In [6]:
# Suppress the UserWarning
warnings.filterwarnings("ignore", category=UserWarning)
# Load the model from the Pickle file
with open("./classifiers/Logistic_Regression.pickle", "rb") as file:
    model_3 = pickle.load(file)


In [7]:
# Suppress the UserWarning
warnings.filterwarnings("ignore", category=UserWarning)
# Load the model from the Pickle file
with open("./classifiers/SVC.pickle", "rb") as file:
    model_4= pickle.load(file)


In [8]:
# Load the model from the Pickle file
with open("./classifiers/AdaBoost.pickle", "rb") as file:
    model_5 = pickle.load(file)


In [9]:
# Suppress the UserWarning
warnings.filterwarnings("ignore", category=UserWarning)
# Load the model from the Pickle file
with open("./classifiers/Gradient_Boosting.pickle", "rb") as file:
    model_6 = pickle.load(file)


In [10]:
def read_test_labels():
    
    test_data_true = []
    y = []
    with open('results_set_1.txt', 'r') as file:
        for line in file:
            y = [float(f) for f in line.split(",")]
            test_data_true.append(y)
    test_data_true = np.asarray(test_data_true)

    return test_data_true

In [11]:
def write_to_file(file_path, variable):
    with open(file_path, 'a') as file:
        file.write(str(variable) + '\n')

In [12]:
def read_test_data():
  all_prediction_1=[]
  all_prediction_2=[]
  all_prediction_3=[]
  all_prediction_4=[]
  all_prediction_5=[]
  all_prediction_6=[]

  all_voting=[]
  main_directory = "./test"
  filenames = os.listdir(main_directory)
  # Define a custom sorting function to extract and sort based on the numbers within filenames
  def sort_by_number(filename):
      numbers = re.findall(r'\d+', filename)  # Extract numbers from the filename
      if numbers:
           return int(numbers[0])  # Convert the first extracted number to an integer
      return 0  # Return 0 if no numbers are found
    
  filenames.sort(key=sort_by_number)  # Sort filenames using the custom sorting function
  # Define a custom sorting function to extract and sort based on the numbers within filenames
  print(filenames)
  for filename in filenames:
      # image_path = main_directory + "/" + filename
      image_path = os.path.join(main_directory, filename)
      # print(image_path)
      image = cv2.imread(image_path)
      start_time = time.time()  # Start the timer
      image=cv2.resize(image, (128, 64))
      roi = preprocess_image(image)
      features=extract_features(roi)
      features_reshaped = features.reshape(1, -1)
      y_pred_1 = int(model_1.predict(features_reshaped)[0])
      y_pred_2=  int(model_2.predict(features_reshaped)[0])
      y_pred_3=  int(model_3.predict(features_reshaped)[0])
      y_pred_4 = int(model_4.predict(features_reshaped)[0])
      y_pred_5=  int(model_5.predict(features_reshaped)[0])
      y_pred_6=  int(model_6.predict(features_reshaped)[0])
  
      all_pred=[y_pred_1,y_pred_2,y_pred_3,y_pred_4,y_pred_5]
      most_frequent = np.bincount(all_pred).argmax()
      all_voting.append(most_frequent)

    #   y_pred_str = ' '.join(str(label) for label in y_pred)
      end_time = time.time()  # Stop the timer
      elapsed_time = round(end_time - start_time, 3)
      write_to_file("results.txt",most_frequent)
      write_to_file("time.txt",elapsed_time)
      all_prediction_1.append(y_pred_1)
      all_prediction_2.append(y_pred_2)
      all_prediction_3.append(y_pred_3)
      all_prediction_4.append(y_pred_4)
      all_prediction_5.append(y_pred_5)
      all_prediction_6.append(y_pred_6)

  return all_prediction_1,all_prediction_2,all_prediction_3,all_prediction_4,all_prediction_5,all_prediction_6,all_voting

In [13]:
# test_trues=read_test_labels()
all_1,all_2,all_3,all_4,all_5,all_6,all_vot=read_test_data()

# accuracy_1 = accuracy_score(test_trues, all_1)
# accuracy_2 = accuracy_score(test_trues, all_2)
# accuracy_3 = accuracy_score(test_trues, all_3)
# accuracy_4 = accuracy_score(test_trues, all_4)
# accuracy_5 = accuracy_score(test_trues, all_5)
# accuracy_6 = accuracy_score(test_trues, all_6)

# accuracy_all_voting = accuracy_score(test_trues, all_vot)

# print(accuracy_1)
# print(accuracy_2)
# print(accuracy_3)
# print(accuracy_4)
# print(accuracy_5)
# print(accuracy_6)

# print(accuracy_all_voting)
# print(all)

['(1).jpg', '(2).jpg', '(3).jpg', '(4).jpg', '(5).jpg', '(6).jpg', '(7).jpg', '(8).jpg', '(9).jpg', '(10).jpg', '(11).jpg', '(12).jpg', '(13).jpg', '(14).jpg', '(15).jpg', '(16).jpg', '(17).jpg', '(18).jpg', '(19).jpg', '(20).jpg', '(21).jpg', '(22).jpg', '(23).jpg', '(24).jpg', '(25).jpg', '(26).jpg', '(27).jpg', '(28).jpg', '(29).jpg', '(30).jpg', '(31).jpg', '(32).jpg', '(33).jpg', '(34).jpg', '(35).jpg', '(36).jpg', '(37).jpg', '(38).jpg', '(39).jpg', '(40).jpg', '(41).jpg', '(42).jpg', '(43).jpg', '(44).jpg', '(45).jpg', '(46).jpg', '(47).jpg', '(48).jpg', '(49).jpg', '(50).jpg', '(51).jpg', '(52).jpg', '(53).jpg', '(54).jpg']
